# 📊 NHS Outpatient Activity Analysis (2012–Present)

This notebook explores quarterly NHS outpatient activity data across England, with a focus on referrals, missed appointments (DNAs), and patient admissions.

The data has been enriched to include:
- Readable quarter labels (e.g. "Q1 2017")
- Calculated DNA rates for first appointments
- GP referral ratios
- Heatmaps and time trends per hospital

This analysis aims to:
- Identify hospitals with high or low performance on key metrics
- Compare GP vs non-GP referral trends over time
- Track missed appointments (DNA) by type and trust
- Provide exportable insights for Power BI dashboards or policy evaluation

In [ ]:
print("Notebook starting...")

import os
print("Available files in working directory:", os.listdir('/kaggle/working'))

# If using uploaded data:
print("Available datasets in input directory:", os.listdir('/kaggle/input'))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

df = pd.read_csv('/kaggle/input/nhs-admissions-data-2011-2020/nhs_cleaned_202504140439.csv')

df.head()

## 🔧 Data Cleaning & Enrichment

We start by parsing the `Period` date column and extracting:
- `fin_year`: NHS financial year
- `fin_q`: Financial quarter (1–4)
- `nhs_quarter`: Label combining quarter and year (e.g. Q3 2019)

We also remove rows where `Provider Org name` is "England", which represents a national total rather than an individual trust.

In [ ]:
# Step 1: Rename 'quarter_start' to 'Period'
df.rename(columns={'quarter_start': 'Period'}, inplace=True)

# Step 2: Reorder so 'Period' is first
cols = df.columns.tolist()
cols = ['Period'] + [col for col in cols if col != 'Period']
df= df[cols]

df.head()

In [ ]:
import seaborn as sns
sns.heatmap(df.isnull(), cbar=False)

checking if there are any null values still in dataset, there are not as i cleaned it in PostgreSQL before importing here 

In [ ]:
df.dtypes  

In [ ]:
df['Period'] = pd.to_datetime(df['Period'])



In [ ]:
df.groupby('Period')['total_admitted'].sum().plot(title="Total Admissions Over Time", figsize=(12,7))

there seems to be an anomalous result in late 2011 with very high admissions 

In [ ]:
df[
    df['Period'].dt.year.isin([2011, 2012,])
].groupby('Period')['total_admitted'].sum()

there is indeed an unusualy large amount for 2011, so break it down by top hospitals;

In [ ]:
df[df['Period'] == '2011-10-01'].groupby('Provider Org name')['total_admitted'].sum().sort_values(ascending=False).head(5)


now we can see the problem is the "england" provider is obviously an aggrigation of all the hopsitals in england, this cell shhould not have been included so we can remove it. 

In [ ]:
df[df['Period'] == '2012-01-01 '].groupby('Provider Org name')['total_admitted'].sum().sort_values(ascending=False).head(5)

In [ ]:
df.groupby('Provider Org name')['total_admitted'].sum().sort_values(ascending=False).head(5)

In [ ]:
df = df[df["Provider Org name"] != "England"]
df.groupby('Provider Org name')['total_admitted'].sum().sort_values(ascending=False).head(5)

In [ ]:
df.groupby('Period')['total_admitted'].sum().plot(title="Total Admissions Over Time", figsize=(12,5))

That looks more realistic, with the admissions slowly increasing over time. the last point in early 2020 is probably low because of incomplete data as it cuts of at some point here. we will not include it in the analysis. 

In [ ]:
df_numeric = df.select_dtypes(include='number')
sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm')

there is not too much to be gained from the relationship matrix as the data is quite basic and directly related to eachother e.g more refferals means more people seen and not attenders

In [ ]:
import matplotlib.pyplot as plt

admissions_by_date = df.groupby('Period')['total_admitted'].sum()

plt.figure(figsize=(12, 5))
plt.plot(admissions_by_date, marker='o')
plt.title("Total Patients Admitted Over Time")
plt.ylabel("Patients Admitted")
plt.xlabel("Quarter")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
referrals_by_date = df.groupby('Period')['total_gp_referrals'].sum()

plt.figure(figsize=(12, 5))
plt.plot(referrals_by_date, color='green')
plt.title("Total GP Referrals Over Time")
plt.ylabel("GP Referrals")
plt.xlabel("Quarter")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
failed_by_date = df.groupby('Period')['total_failed_to_attend'].sum()

plt.figure(figsize=(12, 5))
plt.plot(failed_by_date, color='red')
plt.title("Total DNAs (Failed to Attend) Over Time")
plt.ylabel("Failed to Attend")
plt.xlabel("Quarter")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
agg = df.groupby('Period')[['total_admitted', 'total_gp_referrals', 'total_failed_to_attend']].sum()

plt.figure(figsize=(12, 6))
plt.plot(agg.index, agg['total_admitted'], label='Admissions')
plt.plot(agg.index, agg['total_gp_referrals'], label='GP Referrals')
plt.plot(agg.index, agg['total_failed_to_attend'], label='Failed to Attend')
plt.title("Key NHS Activity Trends (2011–2020)")
plt.xlabel("Quarter")
plt.ylabel("Count")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

📈 1. Prophet Forecast

Model type: Additive time series model
Library: Facebook Prophet

✅ What it does:

Prophet is designed for time series forecasting, especially where there are:
	•	Seasonal patterns (like quarterly NHS cycles)
	•	Long-term trends
	•	Holidays or irregular events

⚙️ What we did:
	•	Used nhs_quarter and total_admitted data
	•	Renamed columns to ds (date) and y (value) as required by Prophet
	•	Fitted a model to historical data (e.g., 2012–2019)
	•	Forecasted admissions for future quarters (e.g., 2020–2023)
	•	Visualized actuals + forecast + uncertainty intervals

📌 Notes:
	•	Prophet assumes regular time intervals (we use financial quarters)
	•	Handles seasonality automatically
	•	Ideal for dashboards and automatic pipelines

⸻



In [ ]:
from prophet import Prophet

#Prepare data

df_prophet = df.groupby('Period'
)['total_admitted'].sum().reset_index() 
df_prophet.columns = ['ds', 'y']

#Fit model

model = Prophet( yearly_seasonality=True, 
                weekly_seasonality=False, 
                daily_seasonality=False,
                changepoint_prior_scale=1.2, # Increase to allow more flexibility in trend 
                seasonality_prior_scale=1, # Increase to allow more wavy patterns
               )
Prophet(changepoint_range=0.5,
                changepoint_prior_scale=0.5) 
model.fit(df_prophet)

#Make forecast

future = model.make_future_dataframe(periods=8, freq='Q') 
forecast = model.predict(future)

#Plot

model.plot(forecast);


🧠 2. XGBoost Forecast

Model type: Tree-based machine learning regression
Library: XGBoost

✅ What it does:

XGBoost is a powerful gradient-boosted decision tree model. It doesn’t require evenly spaced time series and can handle:
	•	Multiple input features (e.g., year, quarter, lags)
	•	Missing data
	•	Non-linear patterns

⚙️ What we did:
	•	Created lag features (e.g., lag_1, lag_4) from total_admitted
	•	Used fin_year, fin_q, lag_1, lag_4 as input features
	•	Trained XGBoost on known data
	•	Predicted values iteratively for future quarters
	•	Compared predictions with historical trends

📌 Notes:
	•	XGBoost needs engineered features (no built-in seasonality)
	•	Often outperforms traditional models in short-term forecasting
	•	Great for stacked models or where external features are added


In [ ]:
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# 🧠 Assign NHS quarter data
def assign_nhs_quarter_data(date):
    month = date.month
    year = date.year
    if 4 <= month <= 6:
        return f"Q1 {year}", year, 1
    elif 7 <= month <= 9:
        return f"Q2 {year}", year, 2
    elif 10 <= month <= 12:
        return f"Q3 {year}", year, 3
    else:
        return f"Q4 {year - 1}", year - 1, 4

# Apply to DataFrame
df[['nhs_quarter', 'fin_year', 'fin_q']] = df['Period'].apply(lambda d: pd.Series(assign_nhs_quarter_data(d)))

# 🧮 Group and sort
df_quarterly = df.groupby(['fin_year', 'fin_q', 'nhs_quarter'])['total_admitted'].sum().reset_index()
df_quarterly = df_quarterly.sort_values(['fin_year', 'fin_q'])

# ❌ Exclude Q4 2019
df_quarterly = df_quarterly[df_quarterly['nhs_quarter'] != 'Q4 2019']

# 🎯 Feature engineering
df_quarterly['lag_1'] = df_quarterly['total_admitted'].shift(1)
df_quarterly['lag_4'] = df_quarterly['total_admitted'].shift(4)

# 🧹 Drop NaNs created by shifting
df_quarterly = df_quarterly.dropna().reset_index(drop=True)

# 🎯 Define features/target
X = df_quarterly[['fin_year', 'fin_q', 'lag_1', 'lag_4']]
y = df_quarterly['total_admitted']
quarter_labels = df_quarterly['nhs_quarter']

# 🔀 Train/test split: last 4 rows as test
X_train, X_test = X[:-4], X[-4:]
y_train, y_test = y[:-4], y[-4:]
test_quarters = quarter_labels[-4:].values

# 🧠 Train XGBoost
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# 🤖 Predict
y_pred = model.predict(X_test)

# 📈 Plot
plt.figure(figsize=(12, 6))

# Historical
plt.plot(quarter_labels[:-4], y_train.values, label='Historical Admissions', marker='o', color='blue')

# Actual
plt.plot(test_quarters, y_test.values, label='Actual (Test)', marker='o', color='green')

# Forecast
plt.plot(test_quarters, y_pred, label='XGBoost Forecast', linestyle='--', marker='x', color='orange')

# 🎨 Labels and layout
plt.title("NHS Admissions by Financial Quarter (w/ XGBoost Forecast)")
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Total Admissions")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

testing the predictive ability using last year of known data 

In [ ]:
# 👷 Your existing cleaned quarterly dataset
df_model = df_quarterly.copy()

# ✅ Step 1: Forecast base — add 8 new rows for 2021–2022
future_quarters = []
last_year = df_model['fin_year'].max()
last_q = df_model[df_model['fin_year'] == last_year]['fin_q'].max()

for i in range(1, 9):  # 8 quarters = 2 years
    next_q = (last_q + i - 1) % 4 + 1
    next_year = last_year + ((last_q + i - 1) // 4)
    quarter_label = f"Q{next_q} {next_year}"
    future_quarters.append({
        'fin_year': next_year,
        'fin_q': next_q,
        'nhs_quarter': quarter_label
    })

df_future = pd.DataFrame(future_quarters)
df_future['total_admitted'] = None  # Placeholder

# ✅ Step 2: Append future to model
df_all = pd.concat([df_model, df_future], ignore_index=True).sort_values(['fin_year', 'fin_q']).reset_index(drop=True)

# ✅ Step 3: Rebuild lag features
df_all['lag_1'] = df_all['total_admitted'].shift(1)
df_all['lag_4'] = df_all['total_admitted'].shift(4)

# ✅ Step 4: Train model on known data
train_df = df_all[df_all['total_admitted'].notnull()]
X_train = train_df[['fin_year', 'fin_q', 'lag_1', 'lag_4']].apply(pd.to_numeric, errors='coerce')
y_train = pd.to_numeric(train_df['total_admitted'], errors='coerce')

from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# ✅ Step 5: Forecast 16 quarters iteratively
for i in range(8):
    idx = train_df.shape[0] + i
    row = df_all.loc[idx]
    X_new = pd.DataFrame([{
        'fin_year': row['fin_year'],
        'fin_q': row['fin_q'],
        'lag_1': df_all.loc[idx - 1, 'total_admitted'],
        'lag_4': df_all.loc[idx - 4, 'total_admitted']
    }])
    X_new = X_new.apply(pd.to_numeric, errors='coerce')
    pred = model.predict(X_new)[0]
    df_all.loc[idx, 'total_admitted'] = pred

# ✅ Step 6: Plot results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# 🔵 Historical (2017–2019)
historical = df_all[df_all['fin_year'].between(2017, 2019)]
plt.plot(historical['nhs_quarter'], historical['total_admitted'], marker='o', label='Historical (2017–2019)', color='blue')

# 🟠 Forecasted (2020–2023)
forecast = df_all[df_all['fin_year'] >= 2020]
plt.plot(forecast['nhs_quarter'], forecast['total_admitted'], marker='x', linestyle='--', label='XGBoost Forecast (2020–2023)', color='orange')

plt.title("NHS Admissions Forecast (2020–2023) with Historical Data")
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Total Admissions")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

forcasting 2 years past end of dataset into 2020-2022 ofcourse the model does not predict covid-19 pandemic so will be considerably off. but the data post 2020 still hasnt been released for we cant see how far of yet. 

🏥 Top 5 Hospitals by Total Admissions

We begin by identifying the top 5 NHS hospitals or providers based on total patient admissions across all quarters in the dataset.

✅ What we did:
	•	Grouped the dataset by Provider Org name and summed total_admitted
	•	Selected the 5 providers with the highest total admissions
	•	Plotted their admission trends over time using a line chart, with one line per hospital

🔍 Why this matters:

In [ ]:
# Step 1: Copy and convert 'Period' to datetime
df = df.copy()
df = df[df['Provider Org name'].str.lower() != 'england']

df['quarter_start'] = pd.to_datetime(df['Period'])

# Step 2: Extract financial year, quarter, and label
df['fin_year'] = df['quarter_start'].dt.year
df['fin_q'] = df['quarter_start'].dt.quarter
df['nhs_quarter'] = 'Q' + df['fin_q'].astype(str) + ' ' + df['fin_year'].astype(str)

# Step 3: Group by hospital and quarter
df_quarterly = df.groupby(
    ['Provider Org name', 'fin_year', 'fin_q', 'nhs_quarter']
)['total_admitted'].sum().reset_index()

# Step 4: Sort for clean plotting
df_quarterly = df_quarterly.sort_values(['Provider Org name', 'fin_year', 'fin_q'])

# Step 5: Filter to top 5 hospitals by total admissions
top_hospitals = df_quarterly.groupby('Provider Org name')['total_admitted'] \
                            .sum().nlargest(5).index
df_top = df_quarterly[df_quarterly['Provider Org name'].isin(top_hospitals)]

# Step 6: Plot admissions over time
plt.figure(figsize=(14, 7))
for org, group in df_top.groupby('Provider Org name'):
    plt.plot(group['nhs_quarter'], group['total_admitted'], marker='o', label=org)

plt.title('Top 5 Hospitals – Admissions Over Time')
plt.xlabel('NHS Financial Quarter')
plt.ylabel('Total Admitted Patients')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(
    title="Hospital",
    loc='upper center',
    bbox_to_anchor=(0.5, -0.15),
    ncol=3,
    fontsize='small'
)
plt.tight_layout()
plt.show()

🔥 Heatmaps of NHS Activity by Hospital and Quarter

To gain a broader view across all providers, we used heatmaps to visualise different metrics over time.

📊 Metrics analysed:
	•	total_admitted: Number of patients admitted
	•	total_failed_to_attend: Missed appointments (all types)
	•	first_dna_rate: DNA rate for first outpatient appointments
	•	gp_referral_rate: Share of referrals that came from GPs

✅ What we did:
	•	Grouped data by Provider Org name and nhs_quarter
	•	Pivoted the data into a matrix format (providers as rows, quarters as columns)
	•	Used Seaborn heatmaps with appropriate colour schemes:
	•	YlGnBu for admissions
	•	Reds or Oranges for DNA rates
	•	Blues for GP referral rates

🎯 Why heatmaps?

Heatmaps reveal patterns over time that would be hard to spot in a single chart:
	•	Hospitals with consistently high DNA rates
	•	Shifts in referral patterns during COVID
	•	Seasonal or annual fluctuations in admissions

In [ ]:
# Step 1: Copy and convert 'Period' to datetime
df = df.copy()
df = df[df['Provider Org name'].str.lower() != 'england']
df['quarter_start'] = pd.to_datetime(df['Period'])

# Step 2: Extract financial year and quarter
df['fin_year'] = df['quarter_start'].dt.year
df['fin_q'] = df['quarter_start'].dt.quarter

# Step 3: Group by hospital and quarter
df_quarterly = df.groupby(
    ['Provider Org name', 'fin_year', 'fin_q']
)['total_admitted'].sum().reset_index()

# Step 4: Create readable labels for the quarter
df_quarterly['nhs_quarter'] = 'Q' + df_quarterly['fin_q'].astype(str) + ' ' + df_quarterly['fin_year'].astype(str)

# Step 5: Create proper column ordering for the heatmap
quarter_order = df_quarterly[['fin_year', 'fin_q', 'nhs_quarter']] \
    .drop_duplicates() \
    .sort_values(['fin_year', 'fin_q'])['nhs_quarter'].tolist()

# Step 6: Pivot data for heatmap
heatmap_data = df_quarterly.pivot(
    index='Provider Org name',
    columns='nhs_quarter',
    values='total_admitted'
)

# Step 6.5: Reorder columns in correct time order
heatmap_data = heatmap_data[[q for q in quarter_order if q in heatmap_data.columns]]

# Step 6.6: Sort rows by total admissions
heatmap_data = heatmap_data.loc[heatmap_data.sum(axis=1).sort_values(ascending=False).index]

# Step 7: Plot heatmap
plt.figure(figsize=(20, 100))
sns.heatmap(
    heatmap_data,
    cmap='YlGnBu',
    linewidths=0.2,
    linecolor='gray'
)

plt.title("NHS Hospital Admissions Heatmap (Chronological)", fontsize=16)
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Provider Organisation")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Step 1: Load and clean the dataset
df = df.copy()
df = df[df['Provider Org name'].str.lower() != 'england']
df['quarter_start'] = pd.to_datetime(df['Period'])

# Step 2: Extract financial year and quarter
df['fin_year'] = df['quarter_start'].dt.year
df['fin_q'] = df['quarter_start'].dt.quarter

# Step 3: Group by hospital and quarter for total_failed_to_attend (DNA)
df_quarterly = df.groupby(
    ['Provider Org name', 'fin_year', 'fin_q']
)['total_failed_to_attend'].sum().reset_index()

# Step 4: Create readable quarter labels
df_quarterly['nhs_quarter'] = 'Q' + df_quarterly['fin_q'].astype(str) + ' ' + df_quarterly['fin_year'].astype(str)

# Step 5: Get quarter order for correct sorting
quarter_order = df_quarterly[['fin_year', 'fin_q', 'nhs_quarter']] \
    .drop_duplicates() \
    .sort_values(['fin_year', 'fin_q'])['nhs_quarter'].tolist()

# Step 6: Pivot data for heatmap
heatmap_data = df_quarterly.pivot(
    index='Provider Org name',
    columns='nhs_quarter',
    values='total_failed_to_attend'
)

# Reorder columns to be in proper quarter order
heatmap_data = heatmap_data[[q for q in quarter_order if q in heatmap_data.columns]]

# Sort hospitals by total DNA volume
heatmap_data = heatmap_data.loc[heatmap_data.sum(axis=1).sort_values(ascending=False).index]

# Step 7: Plot heatmap
plt.figure(figsize=(20, 100))
sns.heatmap(
    heatmap_data,
    cmap='Reds',
    linewidths=0.2,
    linecolor='gray'
)

plt.title("NHS Hospital Failed to Attend (DNA) Heatmap (Chronological)", fontsize=16)
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Provider Organisation")
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Load and clean the dataset
df = df.copy()
df = df[df['Provider Org name'].str.lower() != 'england']
df['quarter_start'] = pd.to_datetime(df['Period'])

# Step 2: Extract financial year and quarter
df['fin_year'] = df['quarter_start'].dt.year
df['fin_q'] = df['quarter_start'].dt.quarter

# Step 3: Group by hospital and quarter for first appointment counts
df_quarterly = df.groupby(
    ['Provider Org name', 'fin_year', 'fin_q']
)[['total_first_seen', 'total_first_dna']].sum().reset_index()

# Step 4: Filter out quarters with fewer than 10 first appointments
df_quarterly = df_quarterly[df_quarterly['total_first_seen'] >= 10]

# Step 5: Calculate first appointment DNA rate
df_quarterly['first_dna_rate'] = df_quarterly['total_first_dna'] / df_quarterly['total_first_seen']

# Step 6: Create readable NHS quarter labels
df_quarterly['nhs_quarter'] = 'Q' + df_quarterly['fin_q'].astype(str) + ' ' + df_quarterly['fin_year'].astype(str)

# Step 7: Build proper quarter order
quarter_order = df_quarterly[['fin_year', 'fin_q', 'nhs_quarter']] \
    .drop_duplicates() \
    .sort_values(['fin_year', 'fin_q'])['nhs_quarter'].tolist()

# Step 8: Pivot to wide format for heatmap
heatmap_data = df_quarterly.pivot(
    index='Provider Org name',
    columns='nhs_quarter',
    values='first_dna_rate'
)

# Step 9: Reorder columns and rows
heatmap_data = heatmap_data[[col for col in quarter_order if col in heatmap_data.columns]]
heatmap_data = heatmap_data.loc[heatmap_data.mean(axis=1).sort_values(ascending=False).index]

# Step 10: Plot the heatmap
plt.figure(figsize=(20, 100))
sns.heatmap(
    heatmap_data,
    cmap='Oranges',
    linewidths=0.2,
    linecolor='gray',
    vmax=1.0
)

plt.title("First Appointment DNA Rate Heatmap by Hospital and Quarter", fontsize=16)
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Provider Organisation")
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Load and clean the dataset
df = df.copy()
df = df[df['Provider Org name'].str.lower() != 'england']
df['quarter_start'] = pd.to_datetime(df['Period'])

# Step 2: Extract financial year and quarter
df['fin_year'] = df['quarter_start'].dt.year
df['fin_q'] = df['quarter_start'].dt.quarter

# Step 3: Group by hospital and quarter for GP and other referrals
df_quarterly = df.groupby(
    ['Provider Org name', 'fin_year', 'fin_q']
)[['total_gp_referrals', 'total_other_referrals']].sum().reset_index()

# Step 4: Filter out quarters with fewer than 10 total referrals
df_quarterly['total_referrals'] = df_quarterly['total_gp_referrals'] + df_quarterly['total_other_referrals']
df_quarterly = df_quarterly[df_quarterly['total_referrals'] >= 10]

# Step 5: Calculate GP referral rate
df_quarterly['gp_referral_rate'] = df_quarterly['total_gp_referrals'] / df_quarterly['total_referrals']

# Step 6: Create readable NHS quarter labels
df_quarterly['nhs_quarter'] = 'Q' + df_quarterly['fin_q'].astype(str) + ' ' + df_quarterly['fin_year'].astype(str)

# Step 7: Build proper quarter order
quarter_order = df_quarterly[['fin_year', 'fin_q', 'nhs_quarter']] \
    .drop_duplicates() \
    .sort_values(['fin_year', 'fin_q'])['nhs_quarter'].tolist()

# Step 8: Pivot to wide format for heatmap
heatmap_data = df_quarterly.pivot(
    index='Provider Org name',
    columns='nhs_quarter',
    values='gp_referral_rate'
)

# Step 9: Reorder columns and sort rows by average GP referral rate
heatmap_data = heatmap_data[[col for col in quarter_order if col in heatmap_data.columns]]
heatmap_data = heatmap_data.loc[heatmap_data.mean(axis=1).sort_values(ascending=False).index]

# Step 10: Plot the heatmap
plt.figure(figsize=(20, 100))
sns.heatmap(
    heatmap_data,
    cmap='Blues',
    linewidths=0.2,
    linecolor='gray',
    vmax=1.0
)

plt.title("GP Referral Rate Heatmap by Hospital and Quarter", fontsize=16)
plt.xlabel("NHS Financial Quarter")
plt.ylabel("Provider Organisation")
plt.tight_layout()
plt.show()

In [ ]:
# Start with the raw dataset
df_export = df.copy()

# Remove the England national summary row (case-insensitive match)
df_export = df_export[df_export['Provider Org name'].str.lower() != 'england']

# Convert the date column
df_export['quarter_start'] = pd.to_datetime(df_export['Period'])

# Add financial year and quarter columns
df_export['fin_year'] = df_export['quarter_start'].dt.year
df_export['fin_q'] = df_export['quarter_start'].dt.quarter
df_export['nhs_quarter'] = 'Q' + df_export['fin_q'].astype(str) + ' ' + df_export['fin_year'].astype(str)

# Export to CSV
df_export.to_csv("nhs_dataset_with_quarters.csv", index=False)